<a href="https://colab.research.google.com/github/ericjenn/working-groups/blob/ericjenn-srpwg-wg1/safety-related-profile/documents/profile_opset/div/div.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

#!pip install onnx onnxruntime


In [2]:
import onnx
import onnxruntime as ort
import numpy as np

# Define input and output tensor names
input_name = "X"
leakyrelu_output_name = "LeakyReluOutput"

#-------------------------------------------------------------------------------
# 2-rank input tensor
#-------------------------------------------------------------------------------

# Create 2-rank input tensor
input_tensor = onnx.helper.make_tensor_value_info(input_name, onnx.TensorProto.FLOAT, [None, None])

# Create output tensor (final result after leakyrelu operation)
leakyrelu_output = onnx.helper.make_tensor_value_info(leakyrelu_output_name, onnx.TensorProto.FLOAT, [None, None])

# Define LeakyRelu node: Y = LeakyRelu(X, alpha)
alpha_value = 0.01  # Example alpha (must be set)
leakyrelu_node = onnx.helper.make_node(
    "LeakyRelu",
    inputs=[input_name],
    outputs=[leakyrelu_output_name],
    alpha=alpha_value
)

# Create the ONNX graph
graph = onnx.helper.make_graph(
    nodes=[leakyrelu_node],
    name="LeakyRelu",
    inputs=[input_tensor],
    outputs=[leakyrelu_output]
)

# Create the ONNX model (LeakyRelu is available in opset 14)
model = onnx.helper.make_model(graph, opset_imports=[onnx.helper.make_opsetid("", 14)])
onnx.checker.check_model(model)

# Save the model
onnx.save(model, "leakyrelu.onnx")

# Load and run the model using ONNX Runtime
session = ort.InferenceSession("leakyrelu.onnx")

def do_leakyrelu(x):
    # Run inference
    output = session.run(None, {input_name: x})

    x_f = (np.array2string(x, separator=',', max_line_width=np.inf).replace('\n', ''))
    o_f = (np.array2string(output[0], separator=',', max_line_width=np.inf).replace('\n', ''))

    # Display results
    print(f"X={x_f}")
    print(f"Result: leakyrelu(X)={o_f}")

np.set_printoptions(precision=None, floatmode='fixed')


#--------------------------------------------------------------------
# Nominal cases
#--------------------------------------------------------------------

# Case N1: 2-rank tensor (float32)
x = np.array([[1.0, -2.0],
              [-3.0, 4.0]], dtype=np.float32)

do_leakyrelu(x)


#--------------------------------------------------------------------
# Non nominal cases (nan, inf, -0, negative values)
#--------------------------------------------------------------------

v = [1.0, 0.0, -1.0, float("inf"), float("-inf"), float("nan"), -0.0]

for x_val in v:
    x_np = np.array([[x_val],
                     [x_val]], dtype=np.float32)
    do_leakyrelu(x_np)


#--------------------------------------------------------------------
# Examples from documentation
#--------------------------------------------------------------------

print("\n## Example 1")

# Example 1 from spec
# X = [6.1, -9.5, 35.7], alpha = 0.1
alpha_value = 0.1
# Recreate model with alpha = 0.1
leakyrelu_node = onnx.helper.make_node(
    "LeakyRelu",
    inputs=[input_name],
    outputs=[leakyrelu_output_name],
    alpha=alpha_value
)
graph = onnx.helper.make_graph(
    nodes=[leakyrelu_node],
    name="LeakyRelu",
    inputs=[input_tensor],
    outputs=[leakyrelu_output]
)
model = onnx.helper.make_model(graph, opset_imports=[onnx.helper.make_opsetid("", 14)])
onnx.checker.check_model(model)
onnx.save(model, "leakyrelu_alpha01.onnx")
session = ort.InferenceSession("leakyrelu_alpha01.onnx")

x_example_1 = np.array([[6.1, -9.5, 35.7]], dtype=np.float32)
do_leakyrelu(x_example_1)


print("\n## Example 2")

# Example 2 from spec
# X = [inf, NaN, -inf, -0.0, 0.0, 1.0, -1.0], alpha = 0.01
alpha_value = 0.01
leakyrelu_node = onnx.helper.make_node(
    "LeakyRelu",
    inputs=[input_name],
    outputs=[leakyrelu_output_name],
    alpha=alpha_value
)
graph = onnx.helper.make_graph(
    nodes=[leakyrelu_node],
    name="LeakyRelu",
    inputs=[input_tensor],
    outputs=[leakyrelu_output]
)
model = onnx.helper.make_model(graph, opset_imports=[onnx.helper.make_opsetid("", 14)])
onnx.checker.check_model(model)
onnx.save(model, "leakyrelu_alpha001.onnx")
session = ort.InferenceSession("leakyrelu_alpha001.onnx")

x_example_2 = np.array([
    [float("inf"), float("nan"), float("-inf"), -0.0, 0.0, 1.0, -1.0]
], dtype=np.float32)
do_leakyrelu(x_example_2)


print("\n## Example 3")

# Example 3 from spec: alpha = NaN
alpha_value = float("nan")
leakyrelu_node = onnx.helper.make_node(
    "LeakyRelu",
    inputs=[input_name],
    outputs=[leakyrelu_output_name],
    alpha=alpha_value
)
graph = onnx.helper.make_graph(
    nodes=[leakyrelu_node],
    name="LeakyRelu",
    inputs=[input_tensor],
    outputs=[leakyrelu_output]
)
model = onnx.helper.make_model(graph, opset_imports=[onnx.helper.make_opsetid("", 14)])
onnx.checker.check_model(model)
onnx.save(model, "leakyrelu_alphanan.onnx")
session = ort.InferenceSession("leakyrelu_alphanan.onnx")

x_example_3 = np.array([
    [float("inf"), float("nan"), float("-inf"), -0.0, 0.0, 1.0, -1.0]
], dtype=np.float32)
do_leakyrelu(x_example_3)


print("\n## Example 4")

# Example 4 from spec: alpha = -inf
alpha_value = float("-inf")
leakyrelu_node = onnx.helper.make_node(
    "LeakyRelu",
    inputs=[input_name],
    outputs=[leakyrelu_output_name],
    alpha=alpha_value
)
graph = onnx.helper.make_graph(
    nodes=[leakyrelu_node],
    name="LeakyRelu",
    inputs=[input_tensor],
    outputs=[leakyrelu_output]
)
model = onnx.helper.make_model(graph, opset_imports=[onnx.helper.make_opsetid("", 14)])
onnx.checker.check_model(model)
onnx.save(model, "leakyrelu_alphainf.onnx")
session = ort.InferenceSession("leakyrelu_alphainf.onnx")

x_example_4 = np.array([
    [float("inf"), float("nan"), float("-inf"), -0.0, 0.0, 1.0, -1.0]
], dtype=np.float32)
do_leakyrelu(x_example_4)


X=[[ 1.00000000,-2.00000000], [-3.00000000, 4.00000000]]
Result: leakyrelu(X)=[[ 1.00000000,-0.02000000], [-0.03000000, 4.00000000]]
X=[[1.00000000], [1.00000000]]
Result: leakyrelu(X)=[[1.00000000], [1.00000000]]
X=[[0.00000000], [0.00000000]]
Result: leakyrelu(X)=[[0.00000000], [0.00000000]]
X=[[-1.00000000], [-1.00000000]]
Result: leakyrelu(X)=[[-0.01000000], [-0.01000000]]
X=[[inf], [inf]]
Result: leakyrelu(X)=[[inf], [inf]]
X=[[-inf], [-inf]]
Result: leakyrelu(X)=[[-inf], [-inf]]
X=[[nan], [nan]]
Result: leakyrelu(X)=[[nan], [nan]]
X=[[-0.00000000], [-0.00000000]]
Result: leakyrelu(X)=[[-0.00000000], [-0.00000000]]

## Example 1
X=[[ 6.09999990,-9.50000000,35.70000076]]
Result: leakyrelu(X)=[[ 6.09999990,-0.94999999,35.70000076]]

## Example 2
X=[[        inf,        nan,       -inf,-0.00000000, 0.00000000, 1.00000000,-1.00000000]]
Result: leakyrelu(X)=[[        inf,        nan,       -inf,-0.00000000, 0.00000000, 1.00000000,-0.01000000]]

## Example 3
X=[[        inf,        nan,